## xml

extensible markup language
여는 태그와 닫는 태그로 구성된 한 쌍
확장 가능
html : hypertext markup language
html로 한계가 있어서 만들어짐
가장 큰 장점 : human- machine-readable
목적도 결국은 데이터 파싱
W3C recommendation
JSON 때문에 사용량 저하
standard way of structuring data
협력해서 작업할 때 약속되어있는 데이터 형태
데이터 스키마를 서로 공유했으면 분업해서 양쪽에서 작업 가능
이런 이름으로 JSON으로 넘길게 하는 약속 (protocol) 하면 ok

둘이 명세서만 맞춰 놓으면 됨.
어떤 키값을 넣을지에 대해서만 맞추면 ok (interface?)

어떤 언어에도 독립적 -> 모든 언어에서 사용 가능

no predifined tag -> 사용자가 모든 태그를 다 정리해서 만들어야 함

텍스트 기반

byte type으로 만들어서 주고받을 수 있음

Validity : validation을 스스로 함 -> 조금이라도 잘못 쓰여 있으면 바로 error
규칙, 규약, 규범 등을 정확히 지켜야만 오류가 없기 때문에 안전!
누가 잘못했는지 명확.

validation과 well-formed는 다른 얘기
형식을 잘 지켜서 만들었다고 validity가 있는 건 아님

### Well-formed Documents
root node가 반드시 있어야 함
계층형 구조를 가지는 정형 데이터
이름은 root가 아님. 최상단에 있다는 뜻
즉 시작하는 태그가 하나는 있어야 함
반드시 여는 태그와 닫는 태그의 쌍으로 이루어져있음
html의 경우 닫는 태그가 없는 경우도 있음 예) meta tag
대소문자 구별 정확하게 해야 함
html은 대소문자 마음대로 써도 상관없음
얘는 대문자로 시작했으면 대문자로 끝나야 하고...무조건 맞춰야 함
attribute들은 각 태그의 안에 포함되어야 함
예) version = "1.0"
값들은 무조건 quote로 싸여 있어야. "가 일반적 '도 사용가능
html보다 훨씬 더 규약적인 측면에서 엄격
-> 오류가 날 확률이 없음
-> validation을 지켜준다는 장점


In [1]:
import xml.etree.ElementTree as et
from lxml import etree

In [4]:
bookstore = et.Element("bookstore")

book1 = et.Element("book", category="cooking")
bookstore.insert(0,book1)

title1 = et.Element("title")
title1.attrib["lang"] = "en"
title1.text = "Everyday Italian"
book1.append(title1)

et.SubElement(book1, "author").text = "Giada De Laurentiis"
et.SubElement(book1, "year").text = "2005"
et.SubElement(book1, "price").text = "30.00"

book2 = et.Element("book", {"category" : "children"})
bookstore.append(book2)

title2 = et.Element("title")
title2.attrib["lang"] = title1.get("lang")
title2.text = "Harry Potter"
book2.append(title2)

et.SubElement(book2, "author").text = "Giada De Laurentiis"
et.SubElement(book2, "year").text = "2005"
et.SubElement(book2, "price").text = "30.00"

et.dump(bookstore)

<bookstore><book category="cooking"><title lang="en">Everyday Italian</title><author>Giada De Laurentiis</author><year>2005</year><price>30.00</price></book><book category="children"><title lang="en">Harry Potter</title><author>Giada De Laurentiis</author><year>2005</year><price>30.00</price></book></bookstore>


In [22]:
root = et.XML(et.tostring(bookstore))

#self.children
print(len(root))
for childNode in root:
    print(childNode.tag, childNode.attrib)

root.clear()

root = et.fromstring(et.tostring(bookstore))

#self.children, list(elem)
childNodes = root.getchildren()
print(len(childNodes))
for childNode in childNodes:
    print(childNode.tag, childNode.items())
    
for childNode in childNodes[0]:
    print(childNode.tag, childNode.keys())
    if childNode.keys() != []:
        print([childNode.get(k) for k in childNode.keys()])
        
book = root.find("book")
print(book.tag, book.get("category"))

bookList = root.findall("book")
for book in bookList:
    print(book.tag, book.get("category"))
    
title = root.find(".//title") # .은 현재위치...root! find는 첫 element만 찾음 findall은 모두 찾음
print(type(title), title.text)

titleList = root.findall(".//title")
print([title.text for title in titleList])

title = root.findtext(".//title")
print(type(title), title)

book = root.find(".//book[@category='children']")
print(book, book.tag)

2
book {'category': 'cooking'}
book {'category': 'children'}
2
book [('category', 'cooking')]
book [('category', 'children')]
title ['lang']
['en']
author []
year []
price []
book cooking
book cooking
book children
<class 'xml.etree.ElementTree.Element'> Everyday Italian
['Everyday Italian', 'Harry Potter']
<class 'str'> Everyday Italian
<Element 'book' at 0x10f9fb9f8> book


/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  del sys.path[0]


In [24]:
book = root.findall(".//book[@category='children']")
print(book)

[<Element 'book' at 0x10f9fb9f8>]


In [26]:
from xml.etree.ElementTree import ElementTree

tree = ElementTree(root)
tree.write("book_xml.xml", encoding = "utf-8", xml_declaration = "utf-8")

In [27]:
from xml.etree.ElementTree import parse

tree = parse("book_xml.xml")
root = tree.getroot()

for node in root.iter():
    print(node.tag, node.text)

bookstore None
book None
title Everyday Italian
author Giada De Laurentiis
year 2005
price 30.00
book None
title Harry Potter
author Giada De Laurentiis
year 2005
price 30.00


In [29]:
tree = ElementTree(file = "book_xml.xml")
root = tree.getroot()

for node in root.iter():
    print(node.tag, node.text)

bookstore None
book None
title Everyday Italian
author Giada De Laurentiis
year 2005
price 30.00
book None
title Harry Potter
author Giada De Laurentiis
year 2005
price 30.00


In [30]:
title = root.find(".//title")
print(type(title), title.text)

<class 'xml.etree.ElementTree.Element'> Everyday Italian


In [31]:
titleList = root.findall(".//title")
print([title.text for title in titleList])

['Everyday Italian', 'Harry Potter']


lxml은 c를 기반으로 만들어짐 속도 빠르다

In [32]:
bookstore = etree.Element("bookstore")

book1 = etree.SubElement(bookstore, "book")
book2 = etree.SubElement(bookstore, "book", attrib = {"category":"children"})

book1.attrib["category"] = "cooking"

title1 = etree.Element("title", lang="en")
title1.text = "Everyday Italian"
book1.append(title1)

etree.SubElement(book1, "author").text = "Giada De Laurentiis"
etree.SubElement(book1, "year").text = "2005"
etree.SubElement(book1, "price").text = "30.00"

title2 = etree.Element("title")
title2.set("lang", title1.get("lang"))
title2.text = "Harry Potter"
book2.append(title2)

etree.SubElement(book2, "author").text = "Giada De Laurentiis"
etree.SubElement(book2, "year").text = "2005"
book2.insert(3, etree.Element("price"))

print(len(book2))
book2[-1].text = "30.00"

# 어디 보내야 할 때
xmlBytes = etree.tostring(bookstore, encoding="utf-8", pretty_print=True, xml_declaration=True)
xmlStr = etree.tounicode(bookstore, pretty_print=True)
#pretty print : tap 들어가게 해서 \n도 해서 출력해줌
print(type(xmlBytes), type(xmlStr)) #<class 'bytes'> <class 'str'>
etree.dump(bookstore)

4
<class 'bytes'> <class 'str'>
<bookstore>
  <book category="cooking">
    <title lang="en">Everyday Italian</title>
    <author>Giada De Laurentiis</author>
    <year>2005</year>
    <price>30.00</price>
  </book>
  <book category="children">
    <title lang="en">Harry Potter</title>
    <author>Giada De Laurentiis</author>
    <year>2005</year>
    <price>30.00</price>
  </book>
</bookstore>


In [33]:
# parsing

xml = etree.XML(etree.tostring(bookstore))
xmlTree = etree.ElementTree(xml)
xmlRoot = xmlTree.getroot()

print(xmlTree.docinfo.xml_version)
print(xmlTree.docinfo.encoding)
print(xmlTree.docinfo.doctype)
print(xmlTree.docinfo.root_name)

print(len(xmlRoot)) # book1, book2 -> 두개
for childNode in xmlRoot:
    print(childNode.tag, childNode.attrib)

1.0
UTF-8

bookstore
2
book {'category': 'cooking'}
book {'category': 'children'}


In [34]:
xml = etree.fromstring(etree.tostring(bookstore))
xmlTree = etree.ElementTree(xml)
xmlRoot = xmlTree.getroot()

childNodes = xmlRoot.getchildren()

print(len(childNodes))
for childNode in childNodes:
    print(childNode.tag, childNode.items())
    
#for childNode in 

2
book [('category', 'cooking')]
book [('category', 'children')]


In [35]:
xmlTree.write("book_tree.xml")
etree.ElementTree(xmlRoot).write("book_root.xml")

In [36]:
xmlTree = etree.parse("book_tree.xml")
xmlRoot = xmlTree.getroot()

etree.dump(xmlRoot)

xmlTree = etree.parse("book_root.xml")
xmlRoot = xmlTree.getroot()

etree.dump(xmlRoot)

<bookstore>
  <book category="cooking">
    <title lang="en">Everyday Italian</title>
    <author>Giada De Laurentiis</author>
    <year>2005</year>
    <price>30.00</price>
  </book>
  <book category="children">
    <title lang="en">Harry Potter</title>
    <author>Giada De Laurentiis</author>
    <year>2005</year>
    <price>30.00</price>
  </book>
</bookstore>
<bookstore>
  <book category="cooking">
    <title lang="en">Everyday Italian</title>
    <author>Giada De Laurentiis</author>
    <year>2005</year>
    <price>30.00</price>
  </book>
  <book category="children">
    <title lang="en">Harry Potter</title>
    <author>Giada De Laurentiis</author>
    <year>2005</year>
    <price>30.00</price>
  </book>
</bookstore>


In [ ]:
xmlObj = etree.fromstring(resStr)